In [ ]:
import matplotlib.pyplot as plt

import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.afw.display as afwDisplay
import lsst.daf.butler as dafButler

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
dataPath = "/repo/main"
collections = ["LATISS/raw/all", "LATISS/calib/u/mreuter/20210611"]
butler = dafButler.Butler(dataPath, collections=collections, instrument="LATISS")

In [ ]:
obs_date = 20200314
seq_num = 332
dataId = {"instrument": "LATISS", "detector": 0,
          "exposure.day_obs": obs_date, "exposure.seq_num": seq_num}
raw = butler.get('raw', dataId)
output_filename = "M87.fits"

In [ ]:
# you only need this in every cell because I'm using %matplotlib notebook. DM Bug??
disp = afwDisplay.Display(1)
disp.scale('asinh', 'zscale')
disp.mtv(raw, title=dataId.items())

In [ ]:
from lsst.ip.isr import AssembleCcdTask

config = AssembleCcdTask.ConfigClass()          # Configuration options
config.doTrim = True
assembleTask = AssembleCcdTask(config=config)   # Build the Task object
exposure = assembleTask.assembleCcd(raw)        # Do the work.

In [ ]:
disp = afwDisplay.Display(1)
disp.scale('asinh', 'zscale')
disp.mtv(exposure, title=dataId.items())

In [ ]:
bias = butler.get('bias', dataId)
dark = butler.get('dark', dataId)
#linearizer = butler.get('linearizer', dataId)
defects = butler.get('defects', dataId)
#flat = butler.get('flat', dataId)

In [ ]:
from lsst.ip.isr import IsrTask
isrConfig = IsrTask.ConfigClass()
isrConfig.doWrite = False
isrConfig.doDark = True
#isrConfig.doSaturation = False
#isrConfig.doSaturationInterpolation = False
#isrConfig.overscanNumLeadingColumnsToSkip = 20
isrConfig.doLinearize = False
isrConfig.doFlat = False
isrTask = IsrTask(config=isrConfig)
#postIsr = isrTask.run(raw, bias=bias, defects=defects).exposure
postIsr = isrTask.run(raw, bias=bias, dark=dark, defects=defects).exposure

In [ ]:
disp = afwDisplay.Display(1)
disp.scale('asinh', 'zscale')
disp.mtv(postIsr, title=dataId.items())

In [ ]:
postIsr.writeFits(output_filename)

In [ ]:
#print(isrConfig)

In [ ]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
imCharConfig = CharacterizeImageTask.ConfigClass()
imCharConfig.doMeasurePsf = True
imCharConfig.doApCorr = False
imCharConfig.doDeblend = False
imCharConfig.repair.cosmicray.nCrPixelMax = 200000
imCharTask = CharacterizeImageTask(config=imCharConfig)
_ = imCharTask.run(postIsr)

In [ ]:
disp = afwDisplay.Display(1)
disp.scale('asinh', 'zscale')
disp.mtv(postIsr, title=dataId.items())